In [57]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import numpy as np
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels, subject_to_info
# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance



import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions
import src.analysis.utils.general_utils as utils # import utils functions one by one by name

In [89]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
subjects = ['D0103']

HOME = os.path.expanduser("~")
USER = os.path.basename(HOME)
    
if os.name == 'nt':  # Windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
elif sys.platform == 'darwin':  # macOS
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
else:  # Linux (cluster)
    # Check if we're on the cluster by looking for /cwork directory
    if os.path.exists(f"/cwork/{USER}"):
        LAB_root = f"/cwork/{USER}"
    else:
        # Fallback for other Linux systems
        LAB_root = os.path.join(HOME, "CoganLab")

config_dir = os.path.join(project_root, 'src', 'analysis', 'config')
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, filename='subjects_electrodestoROIs_dict.json')

task='GlobalLocal'
conditions = experiment_conditions.stimulus_main_effect_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    epochs_root_file = "Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = [condition for condition in conditions.keys()]

if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    
# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from /hpc/group/coganlab/jz421/GlobalLocal/src/analysis/config/subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!
Loading data for subject: D0103
Reading /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/D0103/D0103_Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/D0103/D0103_Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan

/hpc/group/coganlab/jz421/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_ir: 94 valid trials out of 94
  Loading condition: Stimulus_is
Not setting metadata
65 matching events found
No baseline correction applied


/hpc/group/coganlab/jz421/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_is: 65 valid trials out of 65
  Loading condition: Stimulus_cr
Not setting metadata
102 matching events found
No baseline correction applied


...

    Stimulus_is: 65 valid trials out of 65
  Loading condition: Stimulus_cr
Not setting metadata
102 matching events found
No baseline correction applied


/hpc/group/coganlab/jz421/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_cr: 102 valid trials out of 102
  Loading condition: Stimulus_cs
Not setting metadata
97 matching events found
No baseline correction applied


/hpc/group/coganlab/jz421/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_cs: 97 valid trials out of 97


In [90]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
    }

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

Loaded significant channels for subject D0103
For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_f

okay let's try to add previous trial's information

In [91]:
subjects_mne_objects['D0103']['Stimulus_is']['HG_ev1_rescaled']

<EpochsArray | 65 events (all good), -1 – 1.5 s (baseline off), ~70.8 MiB, data loaded,
 'Stimulus/i25.0/s25.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount369.0/BlockTrialCount33.0/ReactionTime866.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s25.0/BigLetters/SmallLetterh/Taskg/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount391.0/BlockTrialCount55.0/ReactionTime716.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s25.0/BigLetters/SmallLetterh/Taskl/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount343.0/BlockTrialCount7.0/ReactionTime1416.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount233.0/BlockTrialCount9.0/ReactionTime1116.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount309.0/BlockTrialCount85.0/ReactionTime1049.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount232.0/BlockTrialCount8.0/ReactionTime1149.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount266.0/BlockTrialCount42.0/ReactionTime1566.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount316.0/BlockTrialCount92.0/ReactionTime1083.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount320.0/BlockTrialCount96.0/ReactionTime1083.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetters/SmallLetterh/Taskg/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount262.0/BlockTrialCount38.0/ReactionTime1900.0/Accuracy1.0/D103': 1
 and 55 more events ...>

In [92]:
import pandas as pd
import numpy as np
import re
import mne

def create_metadata_from_events(epochs):
    """
    Parses MNE hierarchical event IDs to create a metadata DataFrame based 
    on the specific 'GlobalLocal' task structure.
    
    Assumes event names are structured like:
    'Stimulus/i25.0/s75.0/Taskg/ReactionTime866.0/D103'
    
    - part[1] ('i25.0'): Parses congruency and proportion.
    - part[2] ('s75.0'): Parses switch type and proportion.
    - Other parts:
      - 'Taskg' -> key='Task', value='g'
      - 'ReactionTime866.0' -> key='ReactionTime', value='866.0'
      - 'D103' -> key='D', value='103' (This will be fixed)
    """
    
    # Create a reverse mapping from event ID (int) to event name (str)
    id_to_name = {v: k for k, v in epochs.event_id.items()}
    
    # Get the list of integer event IDs for each epoch, in order
    event_ids = epochs.events[:, 2]
    
    all_metadata_rows = []
    
    # Regex for 'i25.0'
    prop_pattern = re.compile(r'([a-zA-Z])([0-9.]+)')
    
    # --- REVERTED ---
    # Use the general key-value parser for *all* other parts.
    # This will correctly parse 'ReactionTime866.0' AND 'D103'.
    other_key_pattern = re.compile(r'([A-Za-z]+)(.+)')

    # Iterate through each epoch's event ID
    for event_id in event_ids:
        full_event_name = id_to_name[event_id]
        parts = full_event_name.split('/')
        row_dict = {}
        
        # --- Parse part 1 (Congruency & Proportion) ---
        if len(parts) > 1:
            match_cong = prop_pattern.match(parts[1])
            if match_cong:
                key, val = match_cong.groups()
                if key == 'i':
                    row_dict['congruency'] = 'incongruent'
                elif key == 'c':
                    row_dict['congruency'] = 'congruent'
                row_dict['incongruent_proportion'] = float(val)
        
        # --- Parse part 2 (Switch Type & Proportion) ---
        if len(parts) > 2:
            match_switch = prop_pattern.match(parts[2])
            if match_switch:
                key, val = match_switch.groups()
                if key == 's':
                    row_dict['switch_type'] = 'switch'
                elif key == 'r':
                    row_dict['switch_type'] = 'repeat'
                else:
                    row_dict['switch_type'] = 'n'
                row_dict['switch_proportion'] = float(val)

        # --- Parse remaining parts (e.g., Task, ReactionTime, D) ---
        for part in parts[3:]: # Start from 3rd part onwards
            match = other_key_pattern.match(part)
            if match:
                key, value = match.groups()
                row_dict[key] = value
        
        all_metadata_rows.append(row_dict)
        
    # Create the DataFrame
    metadata_df = pd.DataFrame(all_metadata_rows)
    
    # --- Post-process the DataFrame ---
    
    # 1. Convert columns to numeric where possible
    metadata_df = metadata_df.apply(pd.to_numeric, errors='ignore')

    # 2. Rename columns
    rename_map = {
        'ReactionTime': 'RT',
        'Accuracy': 'accuracy',
        'D': 'subject'  # --- NEW: Rename 'D' column to 'subject' ---
    }
    
    # Only rename columns that were successfully parsed
    existing_cols_to_rename = {k: v for k, v in rename_map.items() if k in metadata_df.columns}
    metadata_df = metadata_df.rename(columns=existing_cols_to_rename)
    
    # 3. --- FIX: Re-add subject prefix 'D' ---
    # The parser split 'D' (key) and '103' (value). We now recombine them.
    if 'subject' in metadata_df.columns:
        # Convert to integer (to drop any .0), then to string, then prepend 'D'
        metadata_df['subject'] = 'D' + metadata_df['subject'].astype(float).astype(int).astype(str)

    # Attach the new metadata to the epochs object
    epochs.metadata = metadata_df
    
    return epochs

In [93]:
import numpy as np
import pandas as pd

def add_previous_trial_info_to_epochs(epochs):
    """
    Add previous trial information to epochs metadata. Needs a metadata that has all trials for this subject, not just a specific condition.
    
    Parameters:
    -----------
    epochs : mne.Epochs
        The epochs object with metadata
    
    Returns:
    --------
    epochs : mne.Epochs
        Epochs with additional metadata columns for previous trial
    """
    # Make sure we have metadata
    if epochs.metadata is None:
        raise ValueError("Epochs must have metadata")
    
    # Create a copy to avoid modifying original
    metadata = epochs.metadata.copy()
    
    # Add previous trial information
    # Shift by 1 to get previous trial
    metadata['prev_congruency'] = metadata['congruency'].shift(1)
    metadata['prev_switch_type'] = metadata['switch_type'].shift(1)
    metadata['prev_accuracy'] = metadata['accuracy'].shift(1)
    metadata['prev_RT'] = metadata['RT'].shift(1)
    
    # --- Detect block boundaries to set first trial's 'prev_' info to NaN ---
    # A new block is defined by a change in either of the block-level proportions.
    
    # Check for change in incongruent proportion
    if 'incongruent_proportion' in metadata.columns:
        block_starts_cong = metadata['incongruent_proportion'] != metadata['incongruent_proportion'].shift(1)
    else:
        block_starts_cong = pd.Series(False, index=metadata.index)

    # Check for change in switch proportion
    if 'switch_proportion' in metadata.columns:
        block_starts_switch = metadata['switch_proportion'] != metadata['switch_proportion'].shift(1)
    else:
        block_starts_switch = pd.Series(False, index=metadata.index)
        
    # A block starts if *either* proportion changes (or it's the very first trial)
    block_starts = block_starts_cong | block_starts_switch
    
    # Set 'prev_' columns to NaN for all trials that are block starts
    prev_cols = ['prev_congruency', 'prev_switch_type', 'prev_accuracy', 'prev_RT']
    # Filter prev_cols to only include those that actually exist in the metadata
    valid_prev_cols = [col for col in prev_cols if col in metadata.columns]
    metadata.loc[block_starts, valid_prev_cols] = np.nan
    
    # Update the epochs metadata
    epochs.metadata = metadata
    
    return epochs

In [94]:
subjects_mne_objects['D0103']['Stimulus_is']['HG_ev1_rescaled']

<EpochsArray | 65 events (all good), -1 – 1.5 s (baseline off), ~70.8 MiB, data loaded,
 'Stimulus/i25.0/s25.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount369.0/BlockTrialCount33.0/ReactionTime866.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s25.0/BigLetters/SmallLetterh/Taskg/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount391.0/BlockTrialCount55.0/ReactionTime716.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s25.0/BigLetters/SmallLetterh/Taskl/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount343.0/BlockTrialCount7.0/ReactionTime1416.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount233.0/BlockTrialCount9.0/ReactionTime1116.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount309.0/BlockTrialCount85.0/ReactionTime1049.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount232.0/BlockTrialCount8.0/ReactionTime1149.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount266.0/BlockTrialCount42.0/ReactionTime1566.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount316.0/BlockTrialCount92.0/ReactionTime1083.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount320.0/BlockTrialCount96.0/ReactionTime1083.0/Accuracy1.0/D103': 1
 'Stimulus/i25.0/s75.0/BigLetters/SmallLetterh/Taskg/TargetLetters/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount262.0/BlockTrialCount38.0/ReactionTime1900.0/Accuracy1.0/D103': 1
 and 55 more events ...>

In [109]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Load and prepare the master CSV
all_subjects_beh_file = "/hpc/home/jz421/coganlab/jz421/GlobalLocal/combinedData.csv"
master_metadata = pd.read_csv(all_subjects_beh_file)

# Clean up the CSV data
rename_map = {
    'trialCount': 'TrialCount',
    'acc': 'accuracy',
    'subject_ID': 'subject',
    'congruency': 'congruency',
    'switchType': 'switch_type',
    'task': 'Task',
    'blockType': 'blockType'
}
master_metadata = master_metadata.rename(columns=rename_map)

# Map values
master_metadata['congruency'] = master_metadata['congruency'].map(
    {'c': 'congruent', 'i': 'incongruent'}
).fillna(master_metadata['congruency'])

master_metadata['switch_type'] = master_metadata['switch_type'].map(
    {'s': 'switch', 'r': 'repeat', 'n': 'repeat'}
).fillna(master_metadata['switch_type'])

# Sort by subject AND TrialCount
master_metadata = master_metadata.sort_values(by=['subject', 'TrialCount']).reset_index(drop=True)

# Create the lookup table with previous trial info
lookup_df = master_metadata.copy()
trial_cols = ['congruency', 'switch_type', 'accuracy', 'RT']

# Add previous trial columns
for col in trial_cols:
    if col in lookup_df.columns:
        lookup_df[f'prev_{col}'] = lookup_df.groupby('subject')[col].shift(1)

# Handle block boundaries
subject_change = lookup_df['subject'] != lookup_df['subject'].shift(1)
if 'blockType' in lookup_df.columns:
    block_change = lookup_df['blockType'] != lookup_df['blockType'].shift(1)
    block_starts = block_change | subject_change
else:
    block_starts = subject_change

# Set prev columns to NaN at block starts
prev_cols = [f'prev_{col}' for col in trial_cols if col in lookup_df.columns]
lookup_df.loc[block_starts, prev_cols] = np.nan

# Now process each subject's epochs
subjects_mne_objects_with_previous_trial_info = defaultdict(dict)

for sub in subjects:
    print(f"Processing subject: {sub}")
    
    # First, load ALL trials for this subject (unfiltered)
    # You'll need to create or load an epochs object with ALL trials
    # This is the KEY CHANGE - don't filter by condition yet
    
    # Option 1: If you have a way to load all trials at once
    # all_trials_epochs = load_all_trials_for_subject(sub)  # You need to implement this
    
    # Option 2: Reconstruct from your existing filtered epochs
    # For now, let's work with what you have and show the correct approach
    
    for condition_name in conditions.keys():
        epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled']
        
        # Create metadata from events if missing
        if epochs.metadata is None:
            epochs = create_metadata_from_events(epochs)
        
        # Get the subject's data from the lookup table
        subject_lookup = lookup_df[lookup_df['subject'] == sub].copy()
        
        # Merge with the epochs metadata
        epochs_metadata = epochs.metadata.copy()
        
        # Ensure TrialCount is integer
        if 'TrialCount' in epochs_metadata.columns:
            epochs_metadata['TrialCount'] = epochs_metadata['TrialCount'].astype(float).astype(int)
        
        # Drop any existing prev_ columns
        prev_cols_to_drop = [col for col in epochs_metadata.columns if col.startswith('prev_')]
        if prev_cols_to_drop:
            epochs_metadata = epochs_metadata.drop(columns=prev_cols_to_drop)
        
        # Merge to get previous trial info
        # This is the key: we're merging with the COMPLETE behavioral data
        updated_metadata = pd.merge(
            epochs_metadata,
            subject_lookup[['TrialCount'] + prev_cols],
            on='TrialCount',
            how='left'
        )
        
        # Update epochs metadata
        epochs.metadata = updated_metadata
        subjects_mne_objects_with_previous_trial_info[sub][condition_name] = epochs

print("Processing complete.")

# Verify the results
test_metadata = subjects_mne_objects_with_previous_trial_info['D0103']['Stimulus_ir'].metadata
print("\nSample of metadata with previous trial info:")
print(test_metadata[['TrialCount', 'congruency', 'switch_type', 'prev_congruency', 'prev_switch_type']].head(10))
print(f"\nNumber of non-null prev_congruency values: {test_metadata['prev_congruency'].notna().sum()}")
print(f"Total trials: {len(test_metadata)}")

Processing subject: D0103
Replacing existing metadata with 20 columns
Replacing existing metadata with 20 columns
Replacing existing metadata with 20 columns
Replacing existing metadata with 20 columns
Processing complete.

Sample of metadata with previous trial info:
     TrialCount   congruency switch_type prev_congruency prev_switch_type
691           5  incongruent      repeat       congruent           switch
697           7  incongruent      repeat       congruent           repeat
707           8  incongruent      repeat     incongruent           repeat
727           9  incongruent      repeat     incongruent           repeat
731          10  incongruent      repeat     incongruent           repeat
737          11  incongruent      repeat     incongruent           repeat
759          12  incongruent      repeat     incongruent           repeat
771          13  incongruent      repeat     incongruent           repeat
775          14  incongruent      repeat     incongruent         